In [ ]:
import pandas as pd
import numpy as np
import itertools

from tensorflow.python import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Conv2D, Dropout, BatchNormalization, MaxPool2D
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical # convert to one-hot-encoding
from tensorflow.keras.optimizers import RMSprop

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

np.random.seed()
sns.set(style='white', context='notebook', palette='deep')

print("Done")

In [ ]:
# importing custom libraries
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: sys.path.append(dir1)
from graphic_utils.symbols import draw_symbols
from graphic_utils.confusion_matrix import plot_confusion_matrix

# adding support of zip files
from zipfile import ZipFile

In [ ]:
# Load the training data
dataset = pd.read_csv("../data/train.csv.zip", compression='zip')

#Load the test data for the competition submission
competition_dataset = pd.read_csv("../data/test.csv.zip", compression='zip')

dataset.describe()

In [ ]:
# A label is the thing we're predicting
label = dataset["label"]

# A feature is an input variable, in this case a 28 by 28 pixels image
# Drop 'label' column
feature = dataset.drop(labels = ["label"], axis = 1)

# let's check we have a good distribution of the handwritten digits
g = sns.countplot(x=label)

In [ ]:
# free some space
del dataset 

print("Cleaned up")

In [ ]:
draw_symbols(feature, label)


In [ ]:
# Encode labels to one hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])
label = to_categorical(label, num_classes = 10)

# Normalize between 0 and 1 the data (The pixel-value is an integer between 0 and 255)
feature = feature / 255.0
competition_dataset = competition_dataset / 255.0

print("Done")

In [ ]:
# Split the dataset into train and validation set
# Keep 10% for the validation and 90% for the training
# Stratify is argument to keep trainingset evenly balanced ofver the labels (eg validation set not only the digit 5)

feature_train, feature_val, label_train, label_val = train_test_split(feature, label, test_size = 0.1, stratify=label)

In [ ]:
# First model is a dense neural network model with 5 layers
model_1 = Sequential()
model_1.add(Dense(200, activation = "relu", input_shape = (784,)))
model_1.add(Dense(100, activation = "relu"))
model_1.add(Dense(60, activation = "relu"))
model_1.add(Dense(30, activation = "relu"))
model_1.add(Dense(10, activation = "softmax"))

# Define the optimizer and compile the model
optimizer = optimizers.SGD(lr=0.03, clipnorm=5.)
model_1.compile(optimizer= optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

print (model_1.summary())

In [ ]:
# With this model you should be able to achieve around 95.5% accuracy

history = model_1.fit(feature_train, label_train, batch_size = 100, epochs = 8, 
          validation_data = (feature_val, label_val), verbose = 1)

In [ ]:
# Plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
# Predict the values from the validation dataset
Y_pred = model_1.predict(feature_val)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(label_val,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(10)) 

In [ ]:
# predict results
results = model_1.predict(competition_dataset)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

In [ ]:
print(results)